# Normalisation

This is quite an in depth concept and could require an entire section on its own, but as we only need the basics we will cut these down into the key aspects and rationale behind the process.

#### What is Normalisation?

Effectively normalisation is the breaking down of a set of raw data into columns and tables such that we reduce unecessary duplication and redundancy within our data in order to make the processing of this data much more efficient (smaller datasets are easier to ingest and process, for example).

I think this is much easier if we show this in a demo.

First we shall consider the following table:

| EmployeeTitle | EmployeeFName | EmployeeLName | JobTitle | Salary | Department | OrderDate | OrderValue |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Mr | Bob | Smith | Operator | £15000 | TeleSales | 2020-01-01 | £100 |
| Mr | Bob | Smith | Operator | £15000 | TeleSales | 2020-02-01 | £123 |
| Mr | Fred | Jones | Cashier | £14500 | Store Staff | 2020-03-01 | £54 |
| Mr | Bob | Smith | Operator | £14000 | TeleSales | 2020-03-02 | £43 |
| Mr | Fred | Jones | Cashier | £14500 | Store Staff | 2020-03-14 | £83 |
| Mr | Jeff | Baker | Cashier | £14250 | Store Staff | 2020-04-09 | £12 |

There is nothing fundamentally wrong with this data it the format you see it presented here, but in database terms it is less than ideal.

Also consider that this, in the real world, would be a considerably wider table and with potentially millions of records.

For a database engine to store and process this in the best way possible we can immediately see a couple of issues which are the fundamental drivers of Normalisation:
* Duplicate data
* Potential Data Modification Issues
* Simplifying Queries

#### Duplication of Data

We have a finite amount of disk space into which to load our data.  We do not want to be constantly adding more and more disks into our servers (which will have a finite limit they can support) in order to hold duplicate, and therefore redundant, data.  Additionally, the larger our database, the more data we need to read and write to disk which can severly hinder performance.

Databases must also be part of a Backup & Restore (Disaster Recovery) plan and therefore we need to ensure that the database is not unreasonably large which will, by definition, increase the time taken to backup and to restore our databases.

To this end Normalisation suggests that we de-dupe our data, breaking the table into more manageable 'chunks' which become tables in their own rights that we can link back together in order to re-create the original.

#### Potential Data Modification Issues

Imagine we have a table as per the above but considerably wider and with millions of records and we wish to update the job title of anyone listed as 'Cashier' to instead display 'Store Assistant'.

In the current scenario we would have to potentially update hundreds of thousands of records in order to achieve this.  This involves huge amounts of data processing and reource usage for the server, it requires large amounts of logging etc.

Inserting or deleting a new Sale means adding or removing a record from the table, that record being tens of columns wide of textual data which is hard for a database to process and will reduce performance and concurrency.

#### Simplify Queries

If we only wish to see the dates that orders were placed we need to query the entire table, reading all columns and rows.  This is incredibly heavy for SQL Server to perform, using many resources.

Likewise if we wish to see only employees working in the TeleSales department we would, again, have to read the entire length and breadth of the table (which could be huge in the real world) just to return a few records.

## Normalising the Data

To solve these problems we can set about normalising the data that we have in our table.

Firstly we can see if there is any data that naturally groups together, such as:
* Employee Information
 * Employee Title
 * Employee First Name
 * Employee Last Name
 * Salary
* Job Role
 * Job Title
 * Department
* Sales Information
 * Order Date
 * Sales Amount

We will start with our Job Roles:

| JobTitle | Department |
| --- | --- |
| Operator | TeleSales |
| Operator | TeleSales |
| Cashier | Store Staff |
| Operator | TeleSales |
| Cashier | Store Staff |
| Cashier | Store Staff |

Now we can de-dupe this and also add an identifier (which we can then use as a reference in any other tables):

| JobRoleID | JobTitle | Department |
| --- | --- | --- |
| 1 | Operator | TeleSales |
| 2 | Cashier | Store Staff |

Now let's see how this looks for our Employees:

| EmployeeTitle | EmployeeFName | EmployeeLName | Salary |
| --- | --- | --- | --- |
|Mr | Bob | Smith | £15000 |
| Mr | Bob | Smith | £15000 |
| Mr | Fred | Jones | £14500 |
| Mr | Bob | Smith | £14000 |
| Mr | Fred | Jones | £14500 |
| Mr | Jeff | Baker | £14250 |

Now we de-dupe and add an identifier as before, but also include the identifer to link to the Job Role table:

| EmployeeID  | EmployeeTitle | EmployeeFName | EmployeeLName | Salary | JobRoleID |
| --- | --- | --- | --- | --- | --- |
| 1 | Mr | Bob | Smith | £15000 | 1 |
| 2 | Mr | Fred | Jones | £14500 | 2 |
| 3 | Mr | Jeff | Baker | £14250 | 2 |

Our sales data does not need to be de-duplicated as it is different in each record, therefore we simply need to extract it, add an ID, and then include an ID back to our Employee:

| OrderID | EmployeeID | OrderDate | OrderValue |
| --- | --- | --- | --- |
| 1 | 1 | 2020-01-01 | £100 |
| 2 | 1 | 2020-02-01 | £123 |
| 3 | 2 | 2020-03-01 | £54 |
| 4 | 1 | 2020-03-02 | £43 |
| 5 | 2 | 2020-03-14 | £83 |
| 6 | 3 | 2020-04-09 | £12 |

#### Results

We can see that we now have 3 tables which now fit with our normalisation requirements:
* Duplicate data
* Potential Data Modification Issues
* Simplifying Queries

Firstly, our data has been de-duplicated and the space taken up has been reduced.  Again, imagining a million sales and you will see how our normalisation has succeeded as we could have an employee table of just 3 records, a job table of just 2 records, and a sales table of a million rows.  This is a huge saving.

Secondly, we now have cleaner, easier updates and modifications we can perform as an update to a job title now happens in just one table and to one record, the other tables are unaltered as they simlpy reference an ID rather than holding the data itself.

Thirdly, our queries can be much simpler.  For example, if we wish to see the salaries of our employees we now query a 3 record table rather than reading and de-duplicating a several million record table.

Our final tables look as follows:

| JobRoleID | JobTitle | Department |
| --- | --- | --- |
| 1 | Operator | TeleSales |
| 2 | Cashier | Store Staff |

<br>

| EmployeeID  | EmployeeTitle | EmployeeFName | EmployeeLName | Salary | JobRoleID |
| --- | --- | --- | --- | --- | --- |
| 1 | Mr | Bob | Smith | £15000 | 1 |
| 2 | Mr | Fred | Jones | £14500 | 2 |
| 3 | Mr | Jeff | Baker | £14250 | 2 |

<br>

| OrderID | EmployeeID | OrderDate | OrderValue |
| --- | --- | --- | --- |
| 1 | 1 | 2020-01-01 | £100 |
| 2 | 1 | 2020-02-01 | £123 |
| 3 | 2 | 2020-03-01 | £54 |
| 4 | 1 | 2020-03-02 | £43 |
| 5 | 2 | 2020-03-14 | £83 |
| 6 | 3 | 2020-04-09 | £12 |